# Plot Y band resolution vs (dPWV,PWV)

- author Sylvie Dagoret-Campagne
- affiliation IJCLab
- creation date : 2025/11/17
- last update : 2025/11/17
- last update : 2025/11/17 

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import matplotlib as mpl
import matplotlib.colors as colors
import matplotlib.cm as cmx
from mpl_toolkits.axes_grid1 import make_axes_locatable
from matplotlib.colors import LogNorm
from matplotlib.gridspec import GridSpec
import pandas as pd
import h5py

import matplotlib.ticker                         # here's where the formatter is
import os,sys
import re
import pandas as pd

from astropy.io import fits
from astropy import units as u
from astropy import constants as c

plt.rcParams["figure.figsize"] = (8,6)
plt.rcParams["axes.labelsize"] = 'xx-large'
plt.rcParams['axes.titlesize'] = 'xx-large'
plt.rcParams['xtick.labelsize']= 'xx-large'
plt.rcParams['ytick.labelsize']= 'xx-large'

props = dict(boxstyle='round', facecolor='white', alpha=0.5)

In [ ]:
import warnings
warnings.filterwarnings("ignore", category=UserWarning)
warnings.filterwarnings("ignore", category=SyntaxWarning)

In [ ]:
from scipy import interpolate

In [ ]:
machine_name = os.uname().nodename
dm_version = "w_2025_42"
path_rubinsimphot = f"repos/repos_{dm_version}/rubinsimphot/src"
#path_rubinsimphot = "repos/repos_w_2024_17/rubinsimphot/src"
if 'sdf' in machine_name:
    #machine_name_usdf = 'sdfrome001'
    print("Set environment for USDF")
    newpythonpath = os.path.join(os.getenv("HOME"),path_rubinsimphot)
    sys.path.append(newpythonpath)
elif 'dagoret-nb' in machine_name:
    print("Set environment for USDF Rubin Science Platform")
    newpythonpath = os.path.join(os.getenv("HOME"),path_rubinsimphot)
    sys.path.append(newpythonpath)    
elif 'mac' in machine_name:
    print("Be sure to run this notebook in conda environment named conda_py313")
else:
    print(f"Your current machine name is {machine_name}. Check your python environment")

In [ ]:
def GenerateMultiValues(mean,sigma,size, lognorm_flag=True):
    """
    """
    if lognorm_flag:
        mu = np.log(mean**2/np.sqrt(mean**2+sigma**2))
        sig = np.sqrt(np.log(1+sigma**2/mean**2))
        all_values = np.random.lognormal(mean=mu, sigma=sig,size=size)
    else:
        mu = mean
        sig = sigma
        all_values = np.random.normal(mu,sig,size=size)
        
    return all_values

In [ ]:
def CalculateMagsAndMagResolutions(pwv_values, pc,the_sed):
    """
    """

    # compute standard magnitude form the average called std
    mag_std = {}
    adu_std = {}
    atm_bands = pc.bandpass_total_std
    for index,f in enumerate(filter_tagnames) :
        mag_std[f] = the_sed.calc_mag(atm_bands[f])
        adu_std[f] = -2.5*np.log10(the_sed.calc_adu(atm_bands[f],photoparams))

    
    # magnitudes from non stadard pwv
    df = pd.DataFrame(columns = ["pwv","magu","magg","magr","magi","magz","magy","aduu","adug","adur","adui","aduz","aduy"])

    ## loop on pwv values
    for idx_pwv,pwv in enumerate(pwv_values):
        mag_nonstd = {}
        adu_nonstd = {}
        atm_bands = pc.coll_bandpass_total_nonstd[idx_pwv] 

        
        for index,f in enumerate(filter_tagnames) :
            mag_nonstd[f] = the_sed.calc_mag(atm_bands[f])
            adu_nonstd[f] = -2.5*np.log10(the_sed.calc_adu(atm_bands[f],photoparams))

        # add this entry
        df.loc[idx_pwv] = [pwv, mag_nonstd["u"],mag_nonstd["g"],mag_nonstd["r"],mag_nonstd["i"],mag_nonstd["z"],mag_nonstd["y"],
                       adu_nonstd["u"],adu_nonstd["g"],adu_nonstd["r"],adu_nonstd["i"],adu_nonstd["z"],adu_nonstd["y"]] 

    df = df[["pwv","aduu","adug","adur","adui","aduz","aduy"]]

    # compute the magnitude difference
    for index,f in enumerate(filter_tagnames) :
        label_in = f'adu{f}'
        label_out =f'd_adu{f}'
        df[label_out] = (df[label_in]- adu_std[f])*1000. 

    # Drop absolute mags and keep mag difference
    df = df.drop(labels=["aduu","adug","adur","adui","aduz","aduy"],axis=1)


    #compute relative color difference
    df["d_R-I"] = df["d_adur"] -  df["d_adui"]
    df["d_I-Z"] = df["d_adui"] -  df["d_aduz"]
    df["d_Z-Y"] = df["d_aduz"] -  df["d_aduy"]

    return df 

In [ ]:
def MagResolutionFromPWVvsPWV_FromMagResolution(all_PWV_values,all_DPWV_values,nsamples= 1000, am0 = 1, 
                                                oz0 = 300.,tau0=0, beta0=1.2):
    """
    """

    nx = len(all_PWV_values)
    ny = len(all_DPWV_values)
    tresY = np.zeros((ny,nx))
    tresZY = np.zeros((ny,nx))
    
    # loop on PWV values
    for idx,pwv0 in enumerate(all_PWV_values):

        # initialize atmosphere for the typical average conditions pwv0
        pc = PhotometricCorrections(am0,pwv0,oz0,tau0,beta0)


        # create a flat SED and normalize it to have mag 20 in z
        the_sed_flat = Sed()
        the_sed_flat.set_flat_sed()
        the_sed_flat.name = 'flat'
        zmag = 20.0
        flux_norm = the_sed_flat.calc_flux_norm(zmag, pc.bandpass_total_std['z'])
        the_sed_flat.multiply_flux_norm(flux_norm)
    
        # loop on PWV resolution
        for idy,dpwv in enumerate(all_DPWV_values):
            
            # compute the subsamples with varying PWV
            pwv_samples = GenerateMultiValues(pwv0,dpwv, nsamples,lognorm_flag=True)
            pwv_samples= pwv_samples[np.where(np.logical_and(pwv_samples>0., pwv_samples<20.))[0]]
           
            pc.CalculateMultiObs(am0,pwv_samples,oz0,tau0,beta0)

            #compute distribution for magnitude resolution
            df = CalculateMagsAndMagResolutions(pwv_samples, pc, the_sed_flat)
            df_stat = df.describe()
            rms_y = df_stat.loc["std"]["d_aduy"]
            rms_zy = df_stat.loc["std"]["d_Z-Y"]
            
            msg = f"pwv = {pwv0:.1f} mm , dpwv = {dpwv:.3f} mm , res_y = {rms_y:.2f} mmag, res_zy = {rms_zy:.2f} mmag"

            if idy%2 == 0 and idx%2 == 0 :
                print(msg)
                
            tresY[idy,idx] = rms_y
            tresZY[idy,idx] = rms_zy
            
            
    zPWV = am0 * all_PWV_values
    zdPWV = am0 * all_DPWV_values 
                
    return zPWV, zdPWV, tresY, tresZY
        

In [ ]:
def GetdPWVvsPWV_FromMagResolution(all_PWV_values,all_DPWV_values, magresocut=5.0):
    """
    """
    sel_dpwv = []
    sel_df = []
    sel_pwv = []

    # loop on PWV values
    for pwv0 in all_PWV_values:

        # initialize atmosphere for the typical average conditions pwv0
        pc = PhotometricCorrections(am0,pwv0,oz0,tau0,beta0)


        # create a flat SED and noramize it to have mag 20 in z
        the_sed_flat = Sed()
        the_sed_flat.set_flat_sed()
        the_sed_flat.name = 'flat'
        zmag = 20.0
        flux_norm = the_sed_flat.calc_flux_norm(zmag, pc.bandpass_total_std['z'])
        the_sed_flat.multiply_flux_norm(flux_norm)
    
        # loop on PWV resolution
        for dpwv in all_DPWV_values:
            if dpwv >= pwv0:
                continue
                
            # compute the subsamples with varying PWV
            pwv_samples = GenerateMultiValues(pwv0,dpwv, NSAMPLES,lognorm_flag=True)
            pwv_samples= pwv_samples[np.where(np.logical_and(pwv_samples>0., pwv_samples<20.))[0]]
           
            pc.CalculateMultiObs(am0,pwv_samples,oz0,tau0,beta0)

            #compute distribution for magnitude resolution
            df = CalculateMagsAndMagResolutions(pwv_samples, pc, the_sed_flat)
            df_stat = df.describe()
            rms_y = df_stat.loc["std"]["d_aduy"]
            rms_zy = df_stat.loc["std"]["d_Z-Y"]

            if rms_y <=  magresocut: 
                print(f"pwv0 = {pwv0:.3f} mm , dpwv = {dpwv:.3f} mm , rms_y = {rms_y:.2f} mmag rms_z-y = {rms_zy:.2f} mmag")
                sel_dpwv.append(dpwv)
                sel_df.append(df)
                sel_pwv.append(pwv0)
                break
                
    return np.array(sel_pwv), np.array(sel_dpwv), sel_df
        

In [ ]:
# reference flux in Jy
F0 = ((0.*u.ABmag).to(u.Jy)).value
F0

## Imports dedicated to this work

- import the atmospheric transparency emulator (instead of using libradtran code).
- import rubin sim
- import libPhotometricCorrections : encapsulate uninteresting calculation details

### libradtran Emulator

In [ ]:
from importlib.metadata import version
the_ver = version('getObsAtmo')
print(f"Version of getObsAtmo : {the_ver}")

In [ ]:
from getObsAtmo import ObsAtmo
emul = ObsAtmo("LSST")

In [ ]:
WL = emul.GetWL()

#### Library to fit atmosphere

In [ ]:
import sys
sys.path.append('../lib')
#import libAtmosphericFit

#### Library that encapsulate calculations for Photometric correction

In [ ]:
# This package encapsulate the calculation on calibration used in this nb
from libPhotometricCorrections import *

In [ ]:
def set_photometric_parameters(exptime, nexp, readnoise=None):
    # readnoise = None will use the default (8.8 e/pixel). Readnoise should be in electrons/pixel.
    photParams = PhotometricParameters(exptime=exptime, nexp=nexp, readnoise=readnoise)
    return photParams

In [ ]:
def scale_sed(ref_mag, ref_filter, sed):
    fluxNorm = sed.calc_flux_norm(ref_mag, lsst_std[ref_filter])
    sed.multiply_flux_norm(fluxNorm)
    return sed

In [ ]:
# set default photometric parameters to compute ADU
photoparams = set_photometric_parameters(30, 1 , readnoise=None)

#### library rubin_sim defining LSST parameters, namely for photometric calculations

In [ ]:
from rubinsimphot.phot_utils import Bandpass, Sed
from rubinsimphot.data import get_data_dir

## Configuration

In [ ]:
am0 = 1.    # airmass
am_num = int(am0*10.)
pwv0 = 4.0  # Precipitable water vapor vertical column depth in mm
oz0 = 300.  # Ozone vertical column depth in Dobson Unit (DU)
ncomp=1     # Number of aerosol components
tau0= 0.0 # Vertical Aerosol depth (VAOD) 
beta0 = 1.2 # Aerosol Angstrom exponent

### Initialisation of Atmospheric corrections

In [ ]:
pc = PhotometricCorrections(am0,pwv0,oz0,tau0,beta0)

### Check standard atmosphere

In [ ]:
fig, axs = plt.subplots(1,1,figsize=(6,4))
axs.plot(pc.WL,pc.atm_std,'k-')
axs.set_xlabel("$\\lambda$ (nm)")
axs.set_title("Standard atmosphere transmission")

### Check LSST instrument throughput

Photometric Correction package should find the instrumental passband of LSST

In [ ]:
fig, axs = plt.subplots(1,1,figsize=(6,4))
# loop on filter
for index,f in enumerate(filter_tagnames):
    
    axs.plot(pc.bandpass_inst[f].wavelen,pc.bandpass_inst[f].sb,color=filter_color[index]) 
    axs.fill_between(pc.bandpass_inst[f].wavelen,pc.bandpass_inst[f].sb,color=filter_color[index],alpha=0.2) 
    axs.axvline(FILTERWL[index,2],color=filter_color[index],linestyle="-.")
    
axs.set_xlabel("$\\lambda$ (nm)")
axs.set_title("Instrument throughput (auxtel)")

### Check LSST standard Filter throughputs

In [ ]:
fig, axs = plt.subplots(1,1,figsize=(6,4))
# loop on filter
for index,f in enumerate(filter_tagnames):
    
    axs.plot(pc.bandpass_total_std[f].wavelen,pc.bandpass_total_std[f].sb,color=filter_color[index]) 
    axs.fill_between(pc.bandpass_total_std[f].wavelen,pc.bandpass_total_std[f].sb,color=filter_color[index],alpha=0.2) 
    axs.axvline(FILTERWL[index,2],color=filter_color[index],linestyle="-.")
    
axs.set_xlabel("$\\lambda$ (nm)")
axs.set_title("Total filter throughput (auxtel)")

## Distribution of PWV

https://en.wikipedia.org/wiki/Log-normal_distribution


To produce a log-normal distribution with sample mean $\mu_x$ and sample variance $\sigma_x$, use a log-normal distribution with parameters

$$
\mu = \log \frac{\mu_X^2}{\sqrt{\mu_X^2+\sigma_x^2}}
$$

and

$$
\sigma^2 = \log(1+\frac{\sigma_x^2}{\mu_x^2})
$$

In [ ]:
sigma_pwv = .1
pwv0 = 4.0
# Generate a positive distribution of PWV of the desited mam pwv0  and width sigma_pwv
# The log-normal distribution
all_pwv = GenerateMultiValues(pwv0,sigma_pwv,size=1000, lognorm_flag=True)
NPWV = len(all_pwv)

In [ ]:
mu_x = np.mean(all_pwv)
sig_x = np.std(all_pwv)

In [ ]:
textstr = '\n'.join((
    r'$\mu=%.2f$ mm ' % (mu_x, ),
    r'$\sigma=%.2f$ mm' % (sig_x, )))

In [ ]:
fig,ax = plt.subplots(1,1,figsize=(5,4))
ax.hist(all_pwv,bins=50,range=(0,15.),facecolor="b")
ax.text(0.65, 0.95, textstr, transform=ax.transAxes, fontsize=14,verticalalignment='top', bbox=props)
ax.set_title("simulated precipitable water vapor")
ax.set_xlabel("PWV (mm)")

In [ ]:
pc.CalculateMultiObs(am0,all_pwv,oz0,tau0,beta0)

In [ ]:
NOBS = len(all_pwv)

# wavelength bin colors
jet = plt.get_cmap('jet')
cNorm = colors.Normalize(vmin=0, vmax=NOBS)
scalarMap = cmx.ScalarMappable(norm=cNorm, cmap=jet)
all_colors = scalarMap.to_rgba(np.arange(NOBS), alpha=1)


fig = plt.figure(figsize=(6,5))

# Figure 1
axs=fig.add_subplot(1,1,1)
for index,pwv in enumerate(all_pwv):
    atm_bands = pc.coll_bandpass_total_nonstd[index]    
    label = f"pwv={pwv:.1f}" 
    for f in filter_tagnames: 
        axs.plot(atm_bands[f].wavelen,atm_bands[f].sb,color=all_colors[index],label=label,lw=0.5)

axs.plot(pc.WL,pc.atm_std,color="k",lw=2,label="standard atmosphere")
#axs.legend(bbox_to_anchor=(1.03, 1.0))  
axs.set_xlabel("$\lambda$ (nm)")
axs.set_title(f"standard and observed transmission for airmass {am0}")

ax2 = axs.twinx()
for ifilt,f in enumerate(filter_tagnames):
    ax2.fill_between(pc.bandpass_total_std[f].wavelen,pc.bandpass_total_std[f].sb,color=filter_color[ifilt],alpha=0.1) 
    ax2.set_yticks([])


plt.tight_layout()
plt.show()


## Setup rubin sim

In [ ]:
# Find the throughputs directory 
#fdir = os.getenv('RUBIN_SIM_DATA_DIR')
fdir = get_data_dir()
if fdir is None:  #environment variable not set
    fdir = os.path.join(os.getenv('HOME'), 'rubin_sim_data')

## The SED

In [ ]:
the_sed_flat = Sed()
the_sed_flat.set_flat_sed()
the_sed_flat.name = 'flat'
zmag = 20.0
flux_norm = the_sed_flat.calc_flux_norm(zmag, pc.bandpass_total_std['z'])
the_sed_flat.multiply_flux_norm(flux_norm)

In [ ]:
fig,ax = plt.subplots(1,1,figsize=(8,6))
ax.plot(the_sed_flat .wavelen,-2.5*np.log10(the_sed_flat.fnu/F0),"b-",label=the_sed_flat.name)

ax.legend()
#ax.set_ylim(1e-17,1e-14)
#ax.set_xlim(300.,2000.)
ax.set_title("Flat SED $F_\\nu$")
ax.set_ylabel(" Magnitude = $-2.5 \log_{10}(F_\\nu/F_0)$")
ax.set_xlabel("$\\lambda \, (nm)$")
ax.yaxis.set_inverted(True)


ax3 = ax.twinx()
for ifilt,f in enumerate(filter_tagnames):
    ax3.fill_between(pc.bandpass_total_std[f].wavelen,pc.bandpass_total_std[f].sb,color=filter_color[ifilt],alpha=0.1) 
    ax3.set_yticks([])

In [ ]:
the_sed = the_sed_flat

In [ ]:
fig,(ax,ax2) = plt.subplots(1,2,figsize=(16,6))
ax.plot(the_sed .wavelen,the_sed .flambda,"b-",label=the_sed.name)
ax.legend()
ax.set_ylim(1e-17,1e-15)
ax.set_xlim(300.,1200.)
ax.set_title("Flast SED $F_\lambda$")
ax.set_ylabel("$F_\lambda$")
ax.set_xlabel("$\lambda \, (nm)$")


ax2.plot(the_sed .wavelen,the_sed.fnu,"b-",label=the_sed.name)

ax2.set_yscale("log")
ax2.legend()
ax2.set_ylim(1e-5,1e-4)
ax2.set_xlim(300.,1200.)
ax2.set_title("Flat $F_\\nu$")
ax2.set_ylabel("$F_\\nu$")
ax2.set_xlabel("$\lambda \, (nm)$")

ax3 = ax.twinx()
for ifilt,f in enumerate(filter_tagnames):
    ax3.fill_between(pc.bandpass_total_std[f].wavelen,pc.bandpass_total_std[f].sb,color=filter_color[ifilt],alpha=0.1) 
    ax3.set_yticks([])
    
ax4 = ax2.twinx()
for ifilt,f in enumerate(filter_tagnames):
    ax4.fill_between(pc.bandpass_total_std[f].wavelen,pc.bandpass_total_std[f].sb,color=filter_color[ifilt],alpha=0.1) 
    ax4.set_yticks([])

# LOOP on main PWV then on PWV resolution

In [ ]:
#all_PWV = np.arange(1,20.,0.5)
Delta_PWV = 1.0
Delta_PWV_num = int(Delta_PWV*100.)
all_PWV = np.arange(Delta_PWV,20.,Delta_PWV)
print("PWV values",all_PWV) 

In [ ]:
#all_DPWV = np.arange(2.,0.,-0.01)
Delta_dPWV = 0.02
Delta_dPWV_num = int(Delta_dPWV*1000.)
all_DPWV = np.arange(Delta_dPWV,1.,Delta_dPWV)
print(all_DPWV)

In [ ]:
zPWV, zdPWV, tresY, tresZY = MagResolutionFromPWVvsPWV_FromMagResolution(all_PWV,all_DPWV,nsamples= 1000, am0 = 1, 
                                                oz0 = 300.,tau0=0, beta0=1.2)

## Save in file

In [ ]:
filename_out =  f"MagResolutionFromPWVvsPWV_am{am_num}_DPWV{Delta_PWV_num}_DdPWV{Delta_dPWV_num}.h5"

with h5py.File(filename_out, "w") as f:
    f["zPWV"] = zPWV
    f["zdPWV"] = zdPWV
    f["tresY"] = tresY
    f["tresZY"] = tresZY


## Plot

In [ ]:
x = zPWV               # shape (nx,)
y = zdPWV           # shape (ny,)
Z = tresY              # shape (ny, nx)

X, Y = np.meshgrid(x, y)

plt.figure(figsize=(10,8))
pcm = plt.pcolormesh(X, Y, Z, shading='auto',cmap='magma')   # shading='auto' avoids dimension issues

cs = plt.contour(X, Y, Z, levels= np.array([0.5,1.,5.,10.,15.,20.,25.,30.]) ,colors='w', linewidths=0.5)
plt.clabel(cs, inline=True, fontsize=8)

plt.colorbar(pcm, label='$\\sigma(m_y)$ (mmag)')
plt.xlabel('airmass  x PWV (mm)')
plt.ylabel('airmass  x ΔPWV (mm)')
plt.title(f'Magnitude resolution (Y band, airmass = {am0})')

ax = plt.gca()
ax.minorticks_on()
#ax.set_xticks(np.arange(zPWV.min(), zPWV.max(), 0.1))   # 1 mm steps
#ax.set_yticks(np.arange(all_DPWV.min(), all_DPWV.max()+0.01, -0.1))  # 0.1 mm steps

plt.show()

In [ ]:
#y = all_DPWV[::-1]   # reverse it
#Z = tresY[::-1, :]   # reverse rows to stay aligned

In [ ]:
plt.figure(figsize=(10,8))

im = plt.imshow(Z, origin='lower',
           extent=[x.min(), x.max(), y.min(), y.max()],
           aspect='auto',cmap='magma')

cs = plt.contour(X, Y, Z, levels= np.array([0.5,1.,5.,10.,15.,20.,25.,30.]) ,colors='w', linewidths=0.5)
plt.clabel(cs, inline=True, fontsize=8)

plt.colorbar(im,label='$\\sigma(m_y)$ (mmag)')
plt.xlabel('airmass x PWV (mm)')
plt.ylabel('airmass x ΔPWV (mm)')
plt.title(f'Magnitude resolution (Y band, airmass = {am0})')

ax = plt.gca()
ax.minorticks_on()
#ax.set_xticks(np.arange(zPWV.min(), zPWV.max()+1, 1))   # 1 mm steps
#ax.set_yticks(np.arange(all_DPWV.min(), all_DPWV.max()+0.01, -0.1))  # 0.1 mm steps

plt.show()